In [2]:
import zipfile
import os

# Paths to your zip files
zip_paths = [
    "/content/augmented_images.zip",
    "/content/augmented_images(white throated kingfisher).zip",
    "/content/augmented_images(red vented bulbul).zip"
]

# Unzipping function
def unzip_files(zip_paths, extract_to_base):
    for zip_path in zip_paths:
        # Get the folder name based on the zip file name
        folder_name = os.path.splitext(os.path.basename(zip_path))[0]
        extract_to = os.path.join(extract_to_base, folder_name)

        os.makedirs(extract_to, exist_ok=True)
        print(f"Unzipping {zip_path} to {extract_to}...")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Finished unzipping {zip_path} to {extract_to}")

# Unzip all files to separate directories for each bird
extracted_base_dir = "/content/extracted_images"
unzip_files(zip_paths, extracted_base_dir)


Unzipping /content/augmented_images.zip to /content/extracted_images/augmented_images...
Finished unzipping /content/augmented_images.zip to /content/extracted_images/augmented_images
Unzipping /content/augmented_images(white throated kingfisher).zip to /content/extracted_images/augmented_images(white throated kingfisher)...
Finished unzipping /content/augmented_images(white throated kingfisher).zip to /content/extracted_images/augmented_images(white throated kingfisher)
Unzipping /content/augmented_images(red vented bulbul).zip to /content/extracted_images/augmented_images(red vented bulbul)...
Finished unzipping /content/augmented_images(red vented bulbul).zip to /content/extracted_images/augmented_images(red vented bulbul)


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Function to split and organize datasets
def split_and_organize_datasets(base_extracted_dir, split_base_dir, test_size=0.2, val_size=0.2):
    # Clear or create split directories
    if os.path.exists(split_base_dir):
        shutil.rmtree(split_base_dir)
    os.makedirs(split_base_dir, exist_ok=True)

    # Create subdirectories for train, validation, and test
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(split_base_dir, split), exist_ok=True)

    # Loop through each bird folder and split its images
    for bird_folder in os.listdir(base_extracted_dir):
        bird_dir = os.path.join(base_extracted_dir, bird_folder)
        if not os.path.isdir(bird_dir):
            continue

        print(f"Processing {bird_folder}...")

        # Get all image files in the current bird folder
        images = os.listdir(bird_dir)
        images = [img for img in images if img.endswith(('.jpg', '.jpeg', '.png'))]  # Filter valid image files

        if len(images) == 0:
            print(f"No valid image files found in {bird_folder}")
            continue

        # Split the images into train, test, and validation sets
        train_imgs, test_imgs = train_test_split(images, test_size=test_size, random_state=42)
        train_imgs, val_imgs = train_test_split(train_imgs, test_size=val_size, random_state=42)

        # Copy images to the corresponding directories
        for split, imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
            split_dir = os.path.join(split_base_dir, split, bird_folder)
            os.makedirs(split_dir, exist_ok=True)
            for img in imgs:
                shutil.copy(os.path.join(bird_dir, img), os.path.join(split_dir, img))

# Path to your extracted dataset directory
base_extracted_dir = "/content/extracted_images"
split_base_dir = "/content/split_datasets"
split_and_organize_datasets(base_extracted_dir, split_base_dir)


Processing augmented_images...
Processing augmented_images(red vented bulbul)...
Processing augmented_images(white throated kingfisher)...


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Image size and batch size settings
img_size = (224, 224)
batch_size = 32

# Create the ImageDataGenerator objects for train, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Define the directories for train, validation, and test
train_dir = os.path.join("/content/split_datasets", 'train')
val_dir = os.path.join("/content/split_datasets", 'val')
test_dir = os.path.join("/content/split_datasets", 'test')

# Create data generators for train, validation, and test
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 1920 images belonging to 3 classes.
Found 480 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define AlexNet Architecture
def create_alexnet_model(input_shape, num_classes):
    model = Sequential()

    # Layer 1: Conv + ReLU + MaxPooling
    model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 2: Conv + ReLU + MaxPooling
    model.add(Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 3: Conv + ReLU
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'))

    # Layer 4: Conv + ReLU
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'))

    # Layer 5: Conv + ReLU + MaxPooling
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Flattening
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Create the AlexNet model
input_shape = (224, 224, 3)  # Image shape
num_classes = train_gen.num_classes  # Number of bird categories
model = create_alexnet_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

Found 1920 images belonging to 3 classes.
Found 480 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train the model
epochs = 10
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    verbose=1
)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 371s 6s/step - accuracy: 0.3429 - loss: 1.1009 - val_accuracy: 0.4750 - val_loss: 1.0765
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 367s 6s/step - accuracy: 0.4858 - loss: 1.0549 - val_accuracy: 0.5458 - val_loss: 0.9339
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 340s 6s/step - accuracy: 0.6137 - loss: 0.8390 - val_accuracy: 0.6979 - val_loss: 0.6987
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 391s 6s/step - accuracy: 0.8057 - loss: 0.5196 - val_accuracy: 0.8146 - val_loss: 0.4368
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 344s 6s/step - accuracy: 0.8809 - loss: 0.3155 - val_accuracy: 0.8708 - val_loss: 0.3562
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 349s 6s/step - accuracy: 0.9194 - loss: 0.2158 - val_accuracy: 0.8792 - val_loss: 0.3590
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 342s 6s/step - accuracy: 0.9345 - loss: 0.1873 - val_accuracy: 0.8896 - val_loss: 0.3211
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 342s 6s/step - accuracy: 0.9520 - loss: 0.1172 - val_accuracy: 0.9083 - val_loss: 0.

In [7]:
# Calculate the average training accuracy across all epochs
average_training_accuracy = sum(history.history['accuracy']) / len(history.history['accuracy'])

# Print the result
print(f"Average Training Accuracy: {average_training_accuracy:.4f}")


Average Training Accuracy: 0.7931


In [11]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_gen, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate classification report
y_true = test_gen.classes  # Ground truth labels
y_pred = np.argmax(model.predict(test_gen), axis=-1)  # Predicted labels
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))


19/19 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9268 - loss: 0.2650
Test Accuracy: 91.67%
19/19 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step
Classification Report:
                                             precision    recall  f1-score   support

                           augmented_images       0.92      0.95      0.93       200
        augmented_images(red vented bulbul)       0.90      0.89      0.90       200
augmented_images(white throated kingfisher)       0.93      0.91      0.92       200

                                   accuracy                           0.92       600
                                  macro avg       0.92      0.92      0.92       600
                               weighted avg       0.92      0.92      0.92       600

